# OpEn Rust Examples: Various Obstacles

In the previous example, we implemented a path planner for a mobile robot to avoid multiple circular obstacles. For now, we are going to address **other shaped obstacles** including **nonlinear-shaped obstacles**.   


## Features

- We already learnt about **nonlinear-shaped obstacle** formulation in [Example 06](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_obs_avoidance_simplified.ipynb). However, uptil now, we had used a circular obstacle for simplicity.  

- We use **the numerical gradient function** in [Example 07](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_general_diff.ipynb), which allows you not to need analytically obtain the derivatives/gradients of objective functions that you are using. 



## Obstacle Constraints Formulation

### Preliminary: Basic Form

From [Example 06](https://github.com/inmo-jang/optimisation_tutorial/blob/master/tools_examples/OpEn/examples_rust/OpEn_Rust_examples_obs_avoidance_simplified.ipynb), let's remind of that the basic form of obstacle avoidance constraints is as follows:

$$ O = \{ x \in \mathbb{R}^{n_d} : h^i(x) > 0, \forall i \in \mathbb{N}_{[1,m]} \} $$  
<div style="text-align: right"> (1) </div>

where 
- $n_d$: the number of space dimensions (normally 2 or 3)
- $h^i$: the $i$-th nonlinear function that is formed of the obstacle shape. **This function should be designed such that $h^i$ becomes larger than zero as $x$ moves toward inside the obstacle**. 
- $\mathbb{N}_{[1,m]} = \{1,2,...,m\}$: a natural number set from 1 to $m$, which is the number of nonlinear functions for the obstacle.


Let's say $x$ is the position of a mobile robot. In order for the robot to avoid the obstacle, we should make sure the following constraint for every time instant:

$$ x \notin O $$
<div style="text-align: right"> (2) </div>

This constraint is equivalent to 

$$h^i(x) \le 0, \text{for some }i \in \mathbb{N}_{[1,m]} $$
<div style="text-align: right"> (3) </div>

which is reduced to 

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$
<div style="text-align: right"> (4) </div>

For now, we are going to have a look the mathematical formulations for different types of obstacles. 

### (1) Elipsoid

$$h(x) = 1-(x-c)^{\top}E(x-c)$$

where 
- $c \in \mathbb{R}^{n_d}$ is the centre of the elipsoid
- $E \in \mathbb{R}^{n_d \times n_d}$ is a diagonal matrix that controls the size of the elipsoid (i.e. $E = diag(\frac{1}{r_1^2},..., \frac{1}{r^2_{n_d}}$) )

As $x$ is close to $c$, $h(x)$ becomes higher. 

#### 2D Implementation

For 2D, it can be implemented as follows:

In [32]:
// x: user position
// elip : (centre_x, centre_y, radius_x, radius_y)
fn h_elip(x: &[f64], elip: (f64, f64, f64, f64)) -> f64{
    
    
    let h = (1.0 - ((x[0]-elip.0)/elip.2).powi(2) - ((x[1]-elip.1)/elip.3).powi(2) ).max(0.0);  
    
    if h > 0.0 {
        println!("x is inside the obstacle");
    }
    else{
        println!("x is outside the obstacle");
    }
    
    return h;
}


### (2) Polyhedral

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$

where 
$$h^i(x) = b_i - a_i^{\top} x \quad \text{ for each plane $i$ of the object;} $$ 

- $b_i \in \mathbb{R}$: the plane's bias (i.e. distance) from the origin 
- $a_i \in \mathbb{R}^{n_d}$: the outward normal vector of the plane

#### 2D Implementation

In [84]:
// x: user position
// poly : <(a_i, b_i)>
fn h_poly(x: &[f64], poly: Vec<((f64, f64), f64)>, centre: (f64, f64)) -> f64{
    let mut h: f64 = 1.0;
    for i in 0..poly.len(){
        let a_i = poly[i].0;
        let b_i = poly[i].1;
        h *= (b_i - (a_i.0*(x[0] - centre.0) + a_i.1*(x[1]- centre.1))).max(0.0);        
    }
    
    if h > 0.0 {
        println!("x is inside the obstacle");
    }
    else{
        println!("x is outside the obstacle");
    }
    
    return h;
}

### (3) Non-linear Obstacles

The followings are non-linear obtacles used in [Sathya et al., 2019](https://arxiv.org/pdf/1904.10546.pdf) (See Fig 3 in the paper).

$$O_1 = \{y > x^2, y < 1 + \frac{x^2}{2} \} $$

$$O_2 = \{y > 2 \sin(-\frac{x}{2}), y < 3 \sin(\frac{x}{2}-1), 1 < x < 8 \} $$

For now, we are implementing them with addition of their central positions as arguments so that we can adjust their positions. 


Then, the constraint for $O_1$ becomes

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$

where

$$h^1 := (y - c_y) - (x - c_x)^2 > 0$$
$$h^2 := 1 + \frac{(x - c_x)^2}{2} - (y - c_y) > 0$$

In [72]:
fn h_nlr_1(x: &[f64], centre: (f64, f64)) -> f64{
    // x: user position
    // centre : (c_x, c_y)    
    
    let h1 = ((x[1] - centre.1) - ((x[0] - centre.0)).powi(2)).max(0.0);
    let h2 = (1.0 + (x[0] - centre.0).powi(2)/2.0 - (x[1] - centre.1)).max(0.0);
    
    let h = h1*h2;
    
    if h > 0.0 {
        println!("x is inside the obstacle");
    }
    else{
        println!("x is outside the obstacle");
    }
    
    return h;
}

The constraint for $O_2$ becomes

$$\prod_{i=1}^{m} [h^i(x)]^2_{+} = 0$$

where 

$$h_1 := (y - c_y) - 2 \sin(-\frac{x - c_x}{2})$$
$$h_{2-1} := 3\sin(\frac{x-c_x}{2} - 1) - (y-c_y)$$
$$h_{2-2} := x - c_x - 1$$
$$h_{2-3} := 8 - (x - c_x)$$

In [75]:
fn h_nlr_2(x: &[f64], centre: (f64, f64)) -> f64{
    // x: user position
    // centre : (c_x, c_y)    
    
    let h1 = ((x[1] - centre.1) - 2.0*(-(x[0] - centre.0)/2.0).sin()).max(0.0);
    let h2_1 = (3.0*(((x[0] - centre.0)/2.0) - 1.0).sin() - (x[1] - centre.1)).max(0.0);
    let h2_2 = (x[0] - centre.0 - 1.0).max(0.0);
    let h2_3 = (8.0 - (x[0] - centre.0)).max(0.0);
    
    let h = h1*h2_1*h2_2*h2_3;
    
    if h > 0.0 {
        println!("x is inside the obstacle");
    }
    else{
        println!("x is outside the obstacle");
    }
    
    return h;
}

### Test

- Elipsoid: Center $(5, 0)$ and Radius $(2, 1.5)$
- Rectangular: Centre $(0, 0)$, Normal Vectors $\{ (1,0), (-1,0), (0, 1), (0, -1) \}$
- Pentagonal: Centre $(0, 5)$, Normal Vectors $\{ (1,2), (-2,1), (-1, -1), (1, -1), (0, -1) \}$
- Non-linear Object One: Centre $(-5, 0)$
- Non-linear Object Two: Centre $(-5, -5)$

![](https://raw.githubusercontent.com/inmo-jang/optimisation_tutorial/master/tools_examples/OpEn/examples_rust/example_10_nonlinear_obstacles/result.svg)

In [79]:
h_elip(&[3.0, 0.0],(5.0, 0.0, 2.0, 1.5));
h_elip(&[3.01, 0.0],(5.0, 0.0, 2.0, 1.5));

x is outside the obstacle
x is inside the obstacle


In [89]:
let mut poly_A = Vec::new();
poly_A.push(((1.0_f64, 0.0_f64), 1.0_f64));
poly_A.push(((-1.0_f64, 0.0_f64), 1.0_f64));
poly_A.push(((0.0_f64, 1.0_f64), 1.0_f64));
poly_A.push(((0.0_f64, -1.0_f64), 1.0_f64));

In [92]:
h_poly(&[1.0, 1.0], poly_A.clone(), (0.0, 0.0));
h_poly(&[0.999, 0.999], poly_A.clone(), (0.0, 0.0));

x is outside the obstacle
x is inside the obstacle


## Ongoing work....

Soon, we will integrate the path planner example along with the above obstacles. 

**============= The following is not completed yet =============**

## Problem Formulation

Minimise 
$$ f(\mathbf{x}) = \sum_{t=0}^{T_H} (x_t - x_{ref})^{\top}Q_t(x_t - x_{ref}) $$
<div style="text-align: right"> (P1) </div>

where   
- $ \mathbf{x} = \{x_0, x_1, ..., x_{T_H} \} $: the decision variables over the time horizon $T_H$ (i.e. the planned trajectory)
- $x_t \in \mathbb{R}^2$: the 2-d position at time $t$. 
- $x_{ref}$: the goal position. 

subject to 
$$ x_0 = x_{start}$$
<div style="text-align: right"> (C0) </div>

$$ \|x_{t+1} - x_t \| \le u_{max}, \quad\quad k \in \{0,...,T_H-1\}$$
<div style="text-align: right"> (C1) </div>


$$ \psi(\mathbf{x}) =   \sum_{j=1}^{n_{o}}[r_j^2 - (x_t - c_j)^{\top}(x_t - c_j)]_{+} = 0, \quad\quad k \in \{0,...,T_H-1\}$$
<div style="text-align: right"> (C2) </div>

$$ \psi(\mathbf{x}) =   \sum_{j=1}^{n_{o}}[h_j(\mathbf{x})]_{+} = 0, \quad\quad k \in \{0,...,T_H-1\}$$
<div style="text-align: right"> (C2) </div>

$\psi(\mathbf{x})$ becomes $0$ if all $x_t$ is outside every circular shaped obstacle $j$ (nb: $c_j$ is its centre position; $r_j$ is its radius); and increases in the interior of it as $x_t$ moves into the obstacle from its boundary. 

### Reduced Problem

- In this example, we only consider **multiple** circular shaped obstacles.
- We simply consider the time horizon $T_H = 1$, and neglect $Q_t$. 

Thus, the original problem can be reduced to:

Minimise 
$$ f(x_t) = (x_t - x_{ref})^{\top}(x_t - x_{ref}) $$
<div style="text-align: right"> (P1) </div>


subject to 
$$ x_0 = x_{start}$$
<div style="text-align: right"> (C0) </div>

$$ [\|x_{t} - x_0 \| - u_{max}]_{+} = 0$$
<div style="text-align: right"> (C1) </div>


$$ \psi(\mathbf{x}) =   \sum_{j=1}^{n_{o}}[r_j^2 - (x_t - c_j)^{\top}(x_t - c_j)]_{+} = 0, \quad\quad k \in \{0,...,T_H-1\}$$
<div style="text-align: right"> (C2) </div>


## OpEn Implementation

First, we need to import "optimization_engine". Also, we use "nalgebra" for linear algebra calculation. 
- In your local PC, it should be also declared in "Cargo.toml".
- Instead, in this jupyter notebook, we need to have "extern crate" as follows. 


In [2]:
extern crate optimization_engine;
use optimization_engine::{
    alm::*,
    constraints::*, panoc::*, *
};
extern crate nalgebra;
use nalgebra::base::{*};
// use nalgebra::base::{Matrix4, Matrix4x2, Matrix4x1};
// use std::cmp;

### Problem Master Class

You should note that `AlmFactory` should have `f` and `df` being with `
fn f(u: &[f64], cost: &mut f64) -> Result<(), SolverError>` and `fn df(u: &[f64], grad: &mut [f64]) -> Result<(), SolverError>`, respectively. It means that it could be nicer if we have a master class that can simply turn out `f` or `df` values. Such an architecture is used in [`relaxed_ik` rust version](https://github.com/uwgraphics/relaxed_ik/blob/dev/src/RelaxedIK_Rust/src/bin/lib/groove/objective_master.rs), which is a good example to be worth having a look. In this example as well, we are going to implement a problem master class as follows. 


In [3]:
// Problem Master 
pub struct ProblemMaster{
    x_now: Matrix2x1<f64>, // Robot Start Position
    x_ref: Matrix2x1<f64>, // Robot Goal Position
    x_obs: Vec<(f64, f64, f64)>, // Obstacle Position and Radius
    u_max: f64, // Dyanmics Radius
}

impl ProblemMaster{
    pub fn init(_x_start: Matrix2x1<f64>, _x_ref: Matrix2x1<f64>, _x_obs: Vec<(f64, f64, f64)>, _u_max: f64) -> Self {
        let x_now = _x_start;
        let x_ref = _x_ref;
        let x_obs = _x_obs;
        let u_max = _u_max;
        Self{x_now, x_ref, x_obs, u_max}            
    }
    
    
    // ========= Cost function (You need to modify this) =========
    pub fn f_call(&self, u: &[f64]) -> f64{        
        let cost = (u[0]-self.x_ref[(0,0)]).powi(2) + (u[1]-self.x_ref[(1,0)]).powi(2);
        cost
    }
    // ===========================================================
    
    pub fn f(&self, u: &[f64], cost: &mut f64){
        *cost = self.f_call(u);        
    }
    
    // Gradient of the cost function
    pub fn df(&self, u: &[f64], grad: &mut [f64]){
        let f_0 = self.f_call(u);
        
        for i in 0..u.len() {
            let mut u_h = u.to_vec();
            u_h[i] += 0.000001;
            let f_h = self.f_call(u_h.as_slice());
            grad[i] = (-f_0 + f_h) / 0.000001;
        }
       
    } 
    
    // ========= F1 Constraint (You need to modify this) =========
    pub fn f1_call(&self, u: &[f64])-> Vec<f64> {
        let mut f1u = vec![0.0; u.len()];
        // Obstacle Avoidance Constraint (C2)
        let mut f1u_0: f64 = 0.0;        
        for j in 0..self.x_obs.len(){
            f1u_0 += (1.0*(self.x_obs[j].2).powi(2) - (u[0]-self.x_obs[j].0).powi(2) - (u[1]-self.x_obs[j].1).powi(2) ).max(0.0);  
        }
        // f1u[0] = (1.0 - (u[0]-self.x_obs[0].0).powi(2) - (u[1]-self.x_obs[0].1).powi(2) ).max(0.0);        
        // f1u[0] = (1.0 - (u[0]-self.x_obs[(0,0)]).powi(2) - (u[1]-self.x_obs[(1,0)]).powi(2) ).max(0.0);
        f1u[0] = f1u_0;

        // Dynamics Constraint (C1)
        let mut u_now = Matrix2x1::new(0.0, 0.0);
        for i in 0..u.len(){
            u_now[(i,0)] = u[i];
        }
        let delta = u_now - self.x_now;
        f1u[1] = (delta.norm() - self.u_max).max(0.0);
        
        return f1u;
    }
    // ===========================================================
    
    pub fn f1(&self, u: &[f64], f1u: &mut [f64]){
        let f1u_vec = self.f1_call(u); 
        for i in 0..f1u_vec.len(){
            f1u[i] = f1u_vec[i];
        }
    }    
    
    // Jacobian of F1
    pub fn jf1_call(&self, u: &[f64])-> Matrix2<f64> {
        let mut jf1 = Matrix2::new(0.0, 0.0,
                              0.0, 0.0);
        
        let f1_0 = self.f1_call(u);

        for i in 0..f1_0.len(){
            for j in 0..u.len() {
                let mut u_h = u.to_vec();
                u_h[j] += 0.000001;
                let f_h = self.f1_call(u_h.as_slice());
                jf1[(i,j)] = (-f1_0[i] + f_h[i]) / 0.000001;
            }                        
        }

        return jf1;        
    } 
    
    // Jacobian Product (JF_1^{\top}*d)
    pub fn f1_jacobian_product(&self, u: &[f64], d: &[f64], res: &mut [f64]){
        let test = self.f1_call(u);
        
        let mut jf1_matrix = self.jf1_call(u);
        if test[0] < 0.0{ // Outside the obstacle
            jf1_matrix[(0,0)] = 0.0;
            jf1_matrix[(0,1)] = 0.0;  
        }          
        
        let mut d_matrix = Matrix2x1::new(0.0, 0.0);
        for i in 0..d.len(){
            d_matrix[(i,0)] = d[i];
        }
        
        let res_matrix =  jf1_matrix.transpose()*d_matrix;
        
        res[0] = res_matrix[(0,0)];
        res[1] = res_matrix[(1,0)];  
    }
    
}

### Optimisation function for each time step


In [4]:
fn main_loop(_x_now: &[f64], _x_ref: &[f64], _x_obs: Vec<(f64, f64, f64)>, _u_max: f64) -> Vec<f64> {
 
    // ===========================================
    let mut x_now = Matrix2x1::new(0.0, 0.0);
    for i in 0.._x_now.len(){
        x_now[(i,0)] = _x_now[i];
    }
    
    let mut x_ref = Matrix2x1::new(0.0, 0.0);
    for i in 0.._x_ref.len(){
        x_ref[(i,0)] = _x_ref[i];
    }
    
    let x_obs = _x_obs;
    
    let u_max = _u_max;        
    
    let pm = ProblemMaster::init(x_now, x_ref, x_obs, u_max);
    
    // ===========================================
    
    let tolerance = 1e-5;
    let nx = 2; // problem_size: dimension of the decision variables
    let n1 = 2; // range dimensions of mappings F1
    let n2 = 0; // range dimensions of mappings F2
    let lbfgs_mem = 5; // memory of the LBFGS buffer
    
    // PANOCCache: All the information needed at every step of the algorithm
    let panoc_cache = PANOCCache::new(nx, tolerance, lbfgs_mem);
    
    // AlmCache: A cache structure that contains all the data 
    // that make up the state of the ALM/PM algorithm
    // (i.e., all those data that the algorithm updates)
    let mut alm_cache = AlmCache::new(panoc_cache, n1, n2);

    let set_c = Zero::new(); // Set C
    let bounds = Ball2::new(None, 1e12); // Set U
    let set_y = Ball2::new(None, 1e12);  // Set Y

    // ============= 
    // Re-define the functions linked to user parameters
    let f = |u: &[f64], cost: &mut f64| -> Result<(), SolverError> {
        pm.f(u, cost);
        Ok(())
    };
    
    let df = |u: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
        pm.df(u, grad);
        Ok(())
    };
    
    let f1 = |u: &[f64], f1u: &mut [f64]| -> Result<(), SolverError> {
        pm.f1(u, f1u);
        Ok(())
    };    
    
    let f1_jacobian_product = |u: &[f64], d: &[f64], res: &mut [f64]| -> Result<(), SolverError> {
        pm.f1_jacobian_product(u,d,res);
        Ok(())
    };      
    // ==============
    
    // AlmFactory: Prepare function psi and its gradient 
    // given the problem data such as f, del_f and 
    // optionally F_1, JF_1, C, F_2
    let factory = AlmFactory::new(
        f, // Cost function
        df, // Cost Gradient
        Some(f1), // MappingF1
        Some(f1_jacobian_product), // Jacobian Mapping F1 Trans
        NO_MAPPING, // MappingF2
        NO_JACOBIAN_MAPPING, // Jacobian Mapping F2 Trans
        Some(set_c), // Constraint set
        n2,
    );

    // Define an optimisation problem 
    // to be solved with AlmOptimizer
    let alm_problem = AlmProblem::new(
        bounds,
        Some(set_c),
        Some(set_y),
        |u: &[f64], xi: &[f64], cost: &mut f64| -> Result<(), SolverError> {
            factory.psi(u, xi, cost)
        },
        |u: &[f64], xi: &[f64], grad: &mut [f64]| -> Result<(), SolverError> {
            factory.d_psi(u, xi, grad)
        },
        Some(f1),
        NO_MAPPING,
        n1,
        n2,
    );

    let mut alm_optimizer = AlmOptimizer::new(&mut alm_cache, alm_problem)
        .with_delta_tolerance(1e-5)
        .with_max_outer_iterations(200)
        .with_epsilon_tolerance(1e-6)
        .with_initial_inner_tolerance(1e-2)
        .with_inner_tolerance_update_factor(0.5)
        .with_initial_penalty(100.0)
        .with_penalty_update_factor(1.05)
        .with_sufficient_decrease_coefficient(0.2)
        .with_initial_lagrange_multipliers(&vec![5.0; n1]);

    let mut u = _x_now.to_vec(); // Initial guess
    let solver_result = alm_optimizer.solve(&mut u);
    let r = solver_result.unwrap();
//     println!("\n\nSolver result : {:#.7?}\n", r);
//     println!("Solution u = {:#.6?}", u);
    
    return u;
    
}

### Main Function Generating a trajectory 



In [5]:
fn main_instance(_x_now: &[f64], _x_ref: &[f64], _x_obs: Vec<(f64, f64, f64)>, _u_max: f64) -> Vec<(f64, f64)>  {
    
    let mut done = false;
    let mut x_now = _x_now.to_vec();
        
    let mut path_result = Vec::new();
    path_result.push((x_now[0], x_now[1]));
    println!("Solution x_now = {:#.6?}   {:#.6?}", x_now[0], x_now[1]);

    while !done{    
        let x_next = main_loop(x_now.as_slice(), _x_ref, _x_obs.clone(), _u_max);

        x_now = x_next;
        let delta = ((x_now[0]-_x_ref[0]).powi(2)+(x_now[1]-_x_ref[1]).powi(2)).sqrt();
        if delta < 0.00001{
            done = true;
        }
        path_result.push((x_now[0], x_now[1]));
        println!("Solution x_now = {:#.6?}   {:#.6?}", x_now[0], x_now[1]);   
    }
          
    return path_result;
            
        
}

### Result

#### Case (1)
- $x_{start} = [0.0, .0]$;  
- $x_{goal} = [10.0, 10.0]$; 
- $x_{obstacle} = \{(1.0, 2.0, 1.0), (5.0, 4.0, 1.5), (8.0, 7.0, 2.0)\}$; 
    - nb: (pos_x, pos_y, radius)
- $u_{max} = 0.1$. 

In [10]:
let pos_start = &[0.0_f64, 0.0_f64];
let pos_goal = &[10.0_f64, 10.0_f64];
let mut pos_obstacle = Vec::new();
pos_obstacle.push((5.0_f64, 4.0_f64, 1.5_f64)); // (pos_x, pos_y, radius)
pos_obstacle.push((1.0_f64, 2.0_f64, 1.0_f64));
pos_obstacle.push((8.0_f64, 7.0_f64, 2.0_f64));
let max_movement = 0.1_f64;

In [11]:
let path = main_instance(pos_start, pos_goal, pos_obstacle.clone(), max_movement);

Solution x_now = 0.000000   0.000000
Solution x_now = 0.070711   0.070711
Solution x_now = 0.141428   0.141428
Solution x_now = 0.212146   0.212146
Solution x_now = 0.282864   0.282864
Solution x_now = 0.353579   0.353579
Solution x_now = 0.424295   0.424295
Solution x_now = 0.495011   0.495011
Solution x_now = 0.565728   0.565728
Solution x_now = 0.636445   0.636445
Solution x_now = 0.707162   0.707162
Solution x_now = 0.777878   0.777878
Solution x_now = 0.848594   0.848594
Solution x_now = 0.919311   0.919311
Solution x_now = 0.990028   0.990028
Solution x_now = 1.089052   1.003974
Solution x_now = 1.188086   1.017848
Solution x_now = 1.285237   1.041543
Solution x_now = 1.379538   1.074824
Solution x_now = 1.470041   1.117356
Solution x_now = 1.555843   1.168716
Solution x_now = 1.636090   1.228385
Solution x_now = 1.709985   1.295769
Solution x_now = 1.778957   1.368185
Solution x_now = 1.847929   1.440601
Solution x_now = 1.916902   1.513017
Solution x_now = 1.985874   1.585434
S

Solution x_now = 6.013616   7.232984
Solution x_now = 6.013647   7.233246
Solution x_now = 6.014362   7.239260
Solution x_now = 6.014396   7.239539
Solution x_now = 6.014429   7.239812
Solution x_now = 6.014464   7.240102
Solution x_now = 6.015133   7.245578
Solution x_now = 6.015166   7.245836
Solution x_now = 6.015200   7.246118
Solution x_now = 6.015234   7.246390
Solution x_now = 6.015268   7.246662
Solution x_now = 6.015301   7.246926
Solution x_now = 6.015334   7.247194
Solution x_now = 6.015892   7.251630
Solution x_now = 6.015926   7.251901
Solution x_now = 6.015961   7.252174
Solution x_now = 6.015994   7.252432
Solution x_now = 6.016701   7.257930
Solution x_now = 6.016737   7.258206
Solution x_now = 6.017467   7.263749
Solution x_now = 6.018221   7.269358
Solution x_now = 6.018256   7.269618
Solution x_now = 6.020558   7.286031
Solution x_now = 6.021377   7.291643
Solution x_now = 6.038429   7.390178
Solution x_now = 6.043543   7.415062
Solution x_now = 6.049192   7.440850
S

The result can be visualised as follows. 

![](https://raw.githubusercontent.com/inmo-jang/optimisation_tutorial/master/tools_examples/OpEn/examples_rust/example_09_multple_obstacles/path_result.svg)

## Discussion

- [Same as the previous example] The current version planner may not be able to find a collision-avoided path, depending on the given situation. For example, if we start from (0.0, 0.0), the robot would be stuck at around (4.xx, 4.xx), which is the closest point of the obstacle boundary to the start point, without being able to look for any detouring path. That may be because of the current objective function (i.e. norm-2) or the current time horizon being just one time step. 

- **NOTE**: [The source file of this example](https://github.com/inmo-jang/optimisation_tutorial/tree/master/tools_examples/OpEn/examples_rust/example_09_multple_obstacles) is slightly different than here. Using the source file, you can plot the graphs above. 





## Future work

- Let's make the `ProblemMaster` have resizable parameters (e.g. not binding to `Matrix2x1` with consideration of 2D and 3D). 

- Let's consider a longer time horizon. 

- Let's consider a system dynamics. 


